In [1]:
%%writefile model.py
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D,
    BatchNormalization, Activation, Concatenate,
    GlobalAveragePooling2D, Dense, Multiply
)
from tensorflow.keras.models import Model

# -----------------------------
# SE BLOCK
# -----------------------------
def se_block(x, r=16):
    c = x.shape[-1]
    s = GlobalAveragePooling2D()(x)
    s = Dense(c // r, activation="relu")(s)
    s = Dense(c, activation="sigmoid")(s)
    return Multiply()([x, s])

# -----------------------------
# CONV BLOCK
# -----------------------------
def conv_block(x, f):
    x = Conv2D(f, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(f, 3, padding="same")(x)
    x = BatchNormalization()(x)
    return Activation("relu")(x)

# -----------------------------
# MSRF-NET (STABLE)
# -----------------------------
def msrf(input_shape=(256,256,3), edge_shape=(256,256,1)):
    img = Input(input_shape)
    edge = Input(edge_shape)

    # Encoder
    c1 = conv_block(img, 32)
    c1 = se_block(c1)
    p1 = MaxPooling2D()(c1)

    c2 = conv_block(p1, 64)
    c2 = se_block(c2)
    p2 = MaxPooling2D()(c2)

    c3 = conv_block(p2, 128)
    c3 = se_block(c3)

    # Decoder
    u2 = UpSampling2D()(c3)
    u2 = Concatenate()([u2, c2])
    c4 = conv_block(u2, 64)

    u1 = UpSampling2D()(c4)
    u1 = Concatenate()([u1, c1, edge])
    c5 = conv_block(u1, 32)

    # Final output ONLY
    out = Conv2D(1, 1, activation="sigmoid", name="seg")(c5)

    return Model([img, edge], out)

Writing model.py


In [2]:
%%writefile loss.py
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

def get_optimizer():
    return Adam(1e-4)

def _resize_and_cast(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.image.resize(y_pred, tf.shape(y_true)[1:3])
    return y_true, y_pred

def dice_loss(y_true, y_pred):
    y_true, y_pred = _resize_and_cast(y_true, y_pred)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    inter = K.sum(y_true * y_pred)
    return 1 - (2 * inter + 1e-6) / (
        K.sum(y_true) + K.sum(y_pred) + 1e-6
    )

def focal_tversky(y_true, y_pred, a=0.7, b=0.3, g=0.75):
    y_true, y_pred = _resize_and_cast(y_true, y_pred)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    tv = (tp + 1e-6) / (tp + a * fp + b * fn + 1e-6)
    return K.pow((1 - tv), g)

def seg_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + focal_tversky(y_true, y_pred)

def mean_dice(y_true, y_pred):
    y_true, y_pred = _resize_and_cast(y_true, y_pred)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    return (2 * inter) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-6
    )

Writing loss.py


In [31]:
%%writefile metrics.py
import tensorflow as tf

def dice_coef(y_true, y_pred, eps=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    return (2. * inter + eps) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + eps
    )

def iou_coef(y_true, y_pred, eps=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - inter
    return (inter + eps) / (union + eps)

def precision(y_true, y_pred, eps=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    return (tp + eps) / (tp + fp + eps)

def recall(y_true, y_pred, eps=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    return (tp + eps) / (tp + fn + eps)

Writing metrics.py


In [3]:
%%writefile utils.py
import cv2
import numpy as np
from PIL import Image

def clahe_rgb(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l,a,b = cv2.split(lab)
    l = cv2.createCLAHE(2.0,(8,8)).apply(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2RGB)

def remove_hair(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    bh = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT,
                          cv2.getStructuringElement(cv2.MORPH_RECT,(9,9)))
    _, m = cv2.threshold(bh, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, m, 1, cv2.INPAINT_TELEA)

def adaptive_edge(img):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    v = np.median(g)
    e = cv2.Canny(g, int(0.66*v), int(1.33*v))
    return cv2.morphologyEx(e, cv2.MORPH_CLOSE, np.ones((3,3)))

def get_image(path, h, w, gray=False):
    img = Image.open(path).convert("RGB").resize((w,h))
    img = np.array(img)
    img = clahe_rgb(remove_hair(img))
    edge = adaptive_edge(img)

    img = img.astype(np.float32)/255.0
    edge = edge.astype(np.float32)/255.0

    if gray:
        img = (img[...,0] > 0.5).astype(np.float32)

    return img, edge

Writing utils.py


In [20]:
%%writefile prepare_data.py
import os, numpy as np
from glob import glob
from utils import *

BASE = "/kaggle/input/isic2018-challenge-task1-data-segmentation"

X = sorted(glob(f"{BASE}/ISIC2018_Task1-2_Training_Input/*.jpg"))
Y = sorted(glob(f"{BASE}/ISIC2018_Task1_Training_GroundTruth/*.png"))

out = "/kaggle/working/train_cache"
os.makedirs(out, exist_ok=True)

for i,(x,y) in enumerate(zip(X,Y)):
    img, edge = get_image(x,256,256)
    mask,_ = get_image(y,256,256,True)

    np.save(f"{out}/img_{i}.npy", img)
    np.save(f"{out}/edge_{i}.npy", edge[...,None])
    np.save(f"{out}/mask_{i}.npy", mask[...,None])

    if i % 100 == 0:
        print("Processed:", i)

print("Total saved:", i+1)

Overwriting prepare_data.py


In [32]:
%%writefile train_val.py
import tensorflow as tf
import numpy as np
from glob import glob
from model import msrf
from loss import *
from metrics import *

# ======================================================
# CONFIG
# ======================================================
BATCH_SIZE = 8
EPOCHS = 25
VAL_SPLIT = 0.2

# ======================================================
# LOAD FILE PATHS
# ======================================================
imgs  = sorted(glob("/kaggle/working/train_cache/img_*.npy"))
edges = sorted(glob("/kaggle/working/train_cache/edge_*.npy"))
masks = sorted(glob("/kaggle/working/train_cache/mask_*.npy"))

assert len(imgs) > 0
N = len(imgs)

# ======================================================
# SPLIT TRAIN / VAL
# ======================================================
idx = np.arange(N)
np.random.seed(42)
np.random.shuffle(idx)

split = int(N * (1 - VAL_SPLIT))
train_idx, val_idx = idx[:split], idx[split:]

def subset(arr, idx):
    return [arr[i] for i in idx]

train_files = (
    subset(imgs, train_idx),
    subset(edges, train_idx),
    subset(masks, train_idx),
)

val_files = (
    subset(imgs, val_idx),
    subset(edges, val_idx),
    subset(masks, val_idx),
)

print("Train samples:", len(train_idx))
print("Val samples:", len(val_idx))

# ======================================================
# LOADER
# ======================================================
def load_np(img_p, edge_p, mask_p):
    return (
        np.load(img_p.decode()).astype(np.float32),
        np.load(edge_p.decode()).astype(np.float32),
        np.load(mask_p.decode()).astype(np.float32),
    )

def tf_load(img_p, edge_p, mask_p):
    img, edge, mask = tf.numpy_function(
        load_np,
        [img_p, edge_p, mask_p],
        [tf.float32, tf.float32, tf.float32]
    )
    img.set_shape((256,256,3))
    edge.set_shape((256,256,1))
    mask.set_shape((256,256,1))
    return (img, edge), mask

def make_ds(files, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices(files)
    if shuffle:
        ds = ds.shuffle(len(files[0]))
    return (
        ds
        .map(tf_load, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(BATCH_SIZE, drop_remainder=True)
        .prefetch(tf.data.AUTOTUNE)
    )

train_ds = make_ds(train_files, shuffle=True)
val_ds   = make_ds(val_files, shuffle=False)

STEPS_TRAIN = len(train_idx) // BATCH_SIZE
STEPS_VAL   = len(val_idx) // BATCH_SIZE

# ======================================================
# MODEL
# ======================================================
model = msrf()
model.compile(
    optimizer=get_optimizer(),
    loss=seg_loss,
    metrics=[dice_coef, iou_coef, precision, recall]
)

# ======================================================
# TRAIN WITH VALIDATION
# ======================================================
best_val = 0.0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    model.fit(
        train_ds,
        steps_per_epoch=STEPS_TRAIN,
        epochs=1,
        verbose=1
    )

    results = model.evaluate(
        val_ds,
        steps=STEPS_VAL,
        verbose=0
    )

    val_dice = results[1]
    val_iou  = results[2]
    val_prec = results[3]
    val_rec  = results[4]

    print(
        f"Val Dice: {val_dice:.4f} | "
        f"IoU: {val_iou:.4f} | "
        f"Prec: {val_prec:.4f} | "
        f"Recall: {val_rec:.4f}"
    )

    if val_dice > best_val:
        best_val = val_dice
        model.save("best_model.keras")
        print("✅ Best model saved")

print("\nBest Validation Dice:", round(best_val,4))

Overwriting train_val.py


In [5]:
%%writefile test.py
import time, numpy as np, tensorflow as tf
from glob import glob
from utils import *

model = tf.keras.models.load_model("best_model.keras", compile=False)

BASE = "/kaggle/input/isic2018-challenge-task1-data-segmentation"
tests = sorted(glob(f"{BASE}/ISIC2018_Task1-2_Test_Input/*.jpg"))

imgs, edges = [], []
for t in tests:
    i,e = get_image(t,256,256)
    imgs.append(i)
    edges.append(e[...,None])

imgs = np.array(imgs)
edges = np.array(edges)

s = time.time()
pred = model.predict([imgs, edges], batch_size=8)
print("ms/img:", (time.time()-s)/len(imgs)*1000)

print("Lesion %:", pred.mean()*100)
print("Certainty %:", np.mean(np.abs(pred-0.5))*200)

Writing test.py


In [37]:
!ls

best_model.keras  model.py	   test.py	train_val.py
loss.py		  prepare_data.py  train_cache	utils.py
metrics.py	  __pycache__	   train.py


In [38]:
!nvidia-smi

Mon Feb  9 13:38:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             30W /   70W |     121MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [39]:
!ls /kaggle/input/isic2018-challenge-task1-data-segmentation

ISIC2018_Task1-2_Test_Input	 ISIC2018_Task1-2_Validation_Input
ISIC2018_Task1-2_Training_Input  ISIC2018_Task1_Training_GroundTruth


In [21]:
!python prepare_data.py

Processed: 0
Processed: 100
Processed: 200
Processed: 300
Processed: 400
Processed: 500
Processed: 600
Processed: 700
Processed: 800
Processed: 900
Processed: 1000
Processed: 1100
Processed: 1200
Processed: 1300
Processed: 1400
Processed: 1500
Processed: 1600
Processed: 1700
Processed: 1800
Processed: 1900
Processed: 2000
Processed: 2100
Processed: 2200
Processed: 2300
Processed: 2400
Processed: 2500
Total saved: 2594


In [40]:
!ls /kaggle/working
!ls /kaggle/working/train_cache | head
!ls /kaggle/working/train_cache | wc -l

best_model.keras  model.py	   test.py	train_val.py
loss.py		  prepare_data.py  train_cache	utils.py
metrics.py	  __pycache__	   train.py
edge_0.npy
edge_1000.npy
edge_1001.npy
edge_1002.npy
edge_1003.npy
edge_1004.npy
edge_1005.npy
edge_1006.npy
edge_1007.npy
edge_1008.npy
ls: write error: Broken pipe
7782


In [34]:
!python train_val.py

2026-02-09 13:19:48.360419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770643188.380623     872 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770643188.387413     872 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770643188.403122     872 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770643188.403148     872 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770643188.403152     872 computation_placer.cc:177] computation placer alr

In [41]:
!python test.py

2026-02-09 13:38:29.642682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770644309.663794    1616 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770644309.670438    1616 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770644309.688104    1616 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770644309.688131    1616 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770644309.688136    1616 computation_placer.cc:177] computation placer alr